**Import the Requirement Lib**

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2

**Auto Selection of Accelerator (CPU / GPU / TPU)**

In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

    return strategy

In [3]:
#checking the GPU we got from the Kaggle Kernal
!nvidia-smi



#checking the Tensorflow Version
tf.__version__

Wed Nov  9 21:51:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 517.00       Driver Version: 517.00       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:2B:00.0 Off |                  N/A |
| N/A   53C    P0    N/A /  N/A |      0MiB /  2048MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

'2.10.0'

**Pre-Processing the Data**

In [4]:
# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

**Hyper Paramters**

In [5]:
training_set = train_datagen.flow_from_directory('./tomato/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('./tomato/val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 10000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


**Creating the Support Vector Machine Network**

In [6]:
svm = tf.keras.models.Sequential()

# Step 1 - Convolution
svm.add(tf.keras.layers.Conv2D(filters=32,padding="same",kernel_size=3, activation='relu', strides=2, input_shape=[224, 224, 3]))

# Step 2 - Pooling
svm.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
svm.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
svm.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
svm.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
svm.add(tf.keras.layers.Dense(units=128, activation='relu'))

svm.add(Dense(10, kernel_regularizer=tf.keras.regularizers.l2(0.01),activation
             ='softmax'))
svm.compile(optimizer = 'adam', loss = 'squared_hinge', metrics = ['accuracy'])


svm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 56, 56, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 128)               3

**Optimizer and Loss Function**

In [ ]:
svm.compile(optimizer = 'adam', loss = 'squared_hinge', metrics = ['accuracy'])
r=svm.fit(x = training_set, validation_data = test_set, epochs = 1)

**Accuracy Graphs**

In [ ]:
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

**Loading the model**

In [9]:
from tensorflow.keras.models import load_model

svm.save('SVM_Tomatoleaf.h5')

In [10]:
from tensorflow.keras.models import load_model
model = load_model('SVM_Tomatoleaf.h5')

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 56, 56, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 128)               3

In [ ]:
test_set = test_datagen.flow_from_directory('../input/tomato/New Plant Diseases Dataset(Augmented)/train',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

**Testing the Model**

In [54]:

import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img(r"C:\Users\MI\Desktop\AI\Tomato-Leaf-Disease-Detection--main\Tomato-Leaf-Disease-Detection--main\tomato\train\Tomato___Tomato_Yellow_Leaf_Curl_Virus\d01ff1a1-2041-4626-96b4-bcd9ea1a88e5___UF.GRC_YLCV_Lab 01593.JPG", target_size = (224,224))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = svm.predict(test_image)
print(result)

1/1 [==============================] - 0s 72ms/step
[[7.70304359e-06 7.41464873e-07 1.12558143e-15 7.40191297e-10
  4.70427858e-06 2.22487451e-14 3.46403203e-16 9.99986887e-01
  6.99094818e-12 1.09720595e-17]]


In [55]:
import numpy as np
result = np.argmax(result, axis=1)
print(result)


[7]


In [56]:
if result==0:
              print("Bacterial_spot")
elif result==1:
              print("Early_blight")
elif result==2:
              print("Late_blight")
elif result==3:
              print("Leaf_Mold")
elif result==4:
              print("Septoria_leaf_spot")
elif result==5:
              print("Spider_mites Two-spotted_spider_mite")
elif result==6:
              print("Target_Spot")
elif result==7:
              print("Tomato_Yellow_Leaf_Curl_Virus")
elif result==8:
              print("Tomato_mosaic_virus")
else:
              print("Healthy")

Tomato_Yellow_Leaf_Curl_Virus
